###Sneha Malakar
###CMPS 325
###12/13/2021

# Project Two

This project involves observing the progress of several different network architectures learning an image classification task.

Note that each network does require the cells before Part One to have been run, but they do not require the networks before them to have run. So you can break up your work and not have to run all the networks just to re-run the last one.

You should set the runtime type to GPU under the Runtime menu to speed the computations. Note that this may restrict you to having one notebook open at a time.

See a List of TensorFlow datasets at: https://knowyourdata-tfds.withgoogle.com/

See how to load them:
https://www.tensorflow.org/datasets/overview

See data augmentation discussion: https://www.tensorflow.org/tutorials/images/data_augmentation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# TensorFlow
import tensorflow as tf

# Datasets
import tensorflow_datasets as tfds

In [ ]:
# Load in the tf_flowers dataset
# Separate into training, testing and validation sets

train_split = 'train[:80%]'
validation_split = 'train[80%:90%]'
test_split = 'train[90%:]'
(ds_train, ds_validation, ds_test), ds_info = tfds.load(
    'tf_flowers',
    split=[train_split, validation_split, test_split],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

print(ds_info)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]


Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.1. Subsequent calls will reuse this data.
tfds.core.DatasetInfo(
    name='tf_flowers',
    version=3.0.1,
    description='A large set of images of flowers',
    homepage='https://www.tensorflow.org/tutorials/load_data/images',
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=5),
    }),
    total_num_examples=3670,
    splits={
        'train': 3670,
    },
    supervised_keys=('image', 'label'),
    citation="""@ONLINE {tfflowers,
    author = "The TensorFlow Team",
    title = "Flowers",
    month = "jan",
    year = "2019",
    url = "http://download.tensorflow.org/example_images/flower_photos.tgz" }""",
    redistribution_info=,
)



In [ ]:
# Set some network parameters

# Number of classes in the dataset
num_classes = ds_info.features['label'].num_classes
# Size of each image dimension (width and height)
img_size = 80
# Size of a batch of images
batch_size = 128

# Function to apply to each image to prepare it for training and inference
def process_img(image, label):
  # Resize the images, pad if needed to preserve aspect ratio
  #image = tf.image.resize_with_pad(image, img_size, img_size)
  image = tf.image.resize(image, (img_size, img_size))
  # Change from ints in range 0-255 to floats in range 0-1
  image = tf.cast(image, tf.float32) / 255.
  # Change label from integer to one-hot vector
  label = tf.one_hot(label, num_classes)
  return image, label

# Setup data pipeline
# Training images should be shuffled
train = ds_train.map(
    process_img, 
    num_parallel_calls=tf.data.AUTOTUNE
    ).cache().shuffle(ds_info.splits[train_split].num_examples).batch(128).prefetch(tf.data.AUTOTUNE)

# Validation data for tuning hyperparameters
validation = ds_validation.map(
    process_img, 
    num_parallel_calls=tf.data.AUTOTUNE
    ).cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Test data for final evaluation
test = ds_test.map(
    process_img, 
    num_parallel_calls=tf.data.AUTOTUNE
    ).cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)

## Part One - Basic Two-Layer Network

Build the network and the train the model using the cells below.

In [ ]:
# Basic two-layer network
two_layer_model = tf.keras.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(192, activation='relu'),
  tf.keras.layers.Dense(64, activation='relu'),
  tf.keras.layers.Dense(num_classes, activation='softmax'),
])
two_layer_model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
# Train the model
# Watch the training and validation accuracy over time
# A notably higher training accuracy means overfitting
epochs=10
history = two_layer_model.fit(
  train,
  validation_data=validation,
  epochs=epochs
)

Epoch 1/10
23/23 [==============================] - 7s 112ms/step - loss: 9.6443 - categorical_accuracy: 0.2391 - val_loss: 3.8132 - val_categorical_accuracy: 0.2371
Epoch 2/10
23/23 [==============================] - 2s 78ms/step - loss: 3.8576 - categorical_accuracy: 0.2708 - val_loss: 3.5124 - val_categorical_accuracy: 0.2371
Epoch 3/10
23/23 [==============================] - 2s 78ms/step - loss: 3.2659 - categorical_accuracy: 0.3052 - val_loss: 1.6714 - val_categorical_accuracy: 0.3706
Epoch 4/10
23/23 [==============================] - 2s 76ms/step - loss: 2.9938 - categorical_accuracy: 0.3171 - val_loss: 1.9586 - val_categorical_accuracy: 0.3787
Epoch 5/10
23/23 [==============================] - 2s 76ms/step - loss: 2.3091 - categorical_accuracy: 0.3331 - val_loss: 1.9611 - val_categorical_accuracy: 0.3433
Epoch 6/10
23/23 [==============================] - 2s 78ms/step - loss: 2.2538 - categorical_accuracy: 0.3341 - val_loss: 1.7681 - val_categorical_accuracy: 0.3079
Epoch 7/1

In [ ]:
# How big is this model?
two_layer_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 19200)             0         
                                                                 
 dense (Dense)               (None, 192)               3686592   
                                                                 
 dense_1 (Dense)             (None, 64)                12352     
                                                                 
 dense_2 (Dense)             (None, 5)                 325       
                                                                 
Total params: 3,699,269
Trainable params: 3,699,269
Non-trainable params: 0
_________________________________________________________________


### Questions
1. How did the network do? What were the final training and validation accuracies?

The network did not do good because we can see a high values for loss and low values for accuracies. The categorical accuracy is 41.7% whereas the validation categorical accuracy is 35.4%

2. Is there evidence of overfitting?

There is evidence of overfitting because overfitting takes place when the validation accuracy is less then training accuracy. 

3. Why do you think the network had trouble? What are the limitations of this architecture?

I think the network had trouble because the two layer network is not sufficient to handle a non linear data set. Another limitation is the overfitting.

## Part Two - Basic Convolutional Network

Build the network and the train the model using the cells below.

In [ ]:
# A basic convolutional neural network with two sets of convolution/pooling layers
# Notice that typically only one hidden dense layer is used
basic_conv_model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(24, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(192, activation='relu'),
  tf.keras.layers.Dense(num_classes, activation='softmax'),
])
basic_conv_model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
# Train the model
# Watch the training and validation accuracy over time
# A notably higher training accuracy means overfitting
epochs=10
history = basic_conv_model.fit(
  train,
  validation_data=validation,
  epochs=epochs
)

Epoch 1/10
23/23 [==============================] - 15s 624ms/step - loss: 1.9325 - categorical_accuracy: 0.3730 - val_loss: 1.2083 - val_categorical_accuracy: 0.4932
Epoch 2/10
23/23 [==============================] - 14s 619ms/step - loss: 1.2129 - categorical_accuracy: 0.5027 - val_loss: 1.1784 - val_categorical_accuracy: 0.5477
Epoch 3/10
23/23 [==============================] - 14s 619ms/step - loss: 1.1099 - categorical_accuracy: 0.5671 - val_loss: 1.0842 - val_categorical_accuracy: 0.5722
Epoch 4/10
23/23 [==============================] - 14s 616ms/step - loss: 1.0164 - categorical_accuracy: 0.6104 - val_loss: 1.2468 - val_categorical_accuracy: 0.4986
Epoch 5/10
23/23 [==============================] - 14s 618ms/step - loss: 0.9273 - categorical_accuracy: 0.6471 - val_loss: 1.0031 - val_categorical_accuracy: 0.6131
Epoch 6/10
23/23 [==============================] - 14s 618ms/step - loss: 0.8632 - categorical_accuracy: 0.6771 - val_loss: 1.0326 - val_categorical_accuracy: 0.588

In [ ]:
# How big is this model?
basic_conv_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 80, 80, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 40, 40, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 40, 40, 24)        3480      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 20, 20, 24)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 9600)              0         
                                                                 
 dense_3 (Dense)             (None, 192)              

### Questions
1. How did the network do? What were the final training and validation accuracies?

This network worked better than the two layer network. The final training accuracy was 80.6% whereas the final validation accuracy was 53.4%.

2. Is there evidence of overfitting?

There is evidence of overfitting because overfitting takes place when the validation accuracy is less then training accuracy. 

3. What do convolutional networks do to improve learning?

Convolutional networks improve learning by decreasing overfittings and recognizing more number of important features. 

## Augmented Convolutional Neural Network

Build the network and the train the model using the cells below.

In [ ]:
# A convolutional neural network with some features to combat overfitting
# Data augmentation spices up the input by applying random transformations
# Dropout nulls out neurons randomly forcing the network to develop
# redundancy and thus fit less to noise
aug_conv_model = tf.keras.Sequential([
  tf.keras.layers.RandomFlip("horizontal_and_vertical"),
  tf.keras.layers.RandomRotation(0.2),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(24, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(192, activation='relu'),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Dense(num_classes, activation='softmax'),
])
aug_conv_model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

In [ ]:
# Train the model
# Watch the training and validation accuracy over time
# A notably higher training accuracy means overfitting
epochs=10
history = aug_conv_model.fit(
  train,
  validation_data=validation,
  epochs=epochs
)

Epoch 1/10
23/23 [==============================] - 18s 705ms/step - loss: 1.6266 - categorical_accuracy: 0.3559 - val_loss: 1.4601 - val_categorical_accuracy: 0.3270
Epoch 2/10
23/23 [==============================] - 16s 689ms/step - loss: 1.2865 - categorical_accuracy: 0.4445 - val_loss: 1.1738 - val_categorical_accuracy: 0.5259
Epoch 3/10
23/23 [==============================] - 16s 687ms/step - loss: 1.2150 - categorical_accuracy: 0.4894 - val_loss: 1.1198 - val_categorical_accuracy: 0.5886
Epoch 4/10
23/23 [==============================] - 16s 690ms/step - loss: 1.1484 - categorical_accuracy: 0.5283 - val_loss: 1.1877 - val_categorical_accuracy: 0.5286
Epoch 5/10
23/23 [==============================] - 16s 689ms/step - loss: 1.1081 - categorical_accuracy: 0.5565 - val_loss: 1.1589 - val_categorical_accuracy: 0.5477
Epoch 6/10
23/23 [==============================] - 16s 685ms/step - loss: 1.0746 - categorical_accuracy: 0.5613 - val_loss: 1.0154 - val_categorical_accuracy: 0.640

In [ ]:
# Evaluate its performance on the test set
print("Testing Results:")
aug_conv_model.evaluate(test)

Testing Results:
3/3 [==============================] - 1s 244ms/step - loss: 0.8811 - categorical_accuracy: 0.6294


[0.8810797929763794, 0.6294277906417847]

### Questions
1. How did the network do? What were the final training and validation accuracies?

The network did better than the two layer network but worse than the basic convolutional network. The final training accuracy was 61.6% and validation accuracy was 65.1%

2. Is there evidence of overfitting?

There is evidence of overfitting because overfitting takes place when the validation accuracy is less then training accuracy. 

3. Is the overfitting reduced compared to the previous network?

Yes.

4. How did the network do on the test data that it hadn't seen before? Was the accuracy comparable?

The test data had a loss of 88.1% and testing accuracy of 62.9%. The training accuracy and the testing accuracy are pretty close. 

## Network with Pre-trained Feature Extractors

Build the network and the train the model using the cells below.

In [ ]:
# Load in the pre-trained model
# Note that include_top=False means we are including just
# the feature extractors and not the classification layers
pre_trained_model = tf.keras.applications.inception_v3.InceptionV3(
    input_shape=(img_size, img_size, 3), include_top=False)

# Make sure we don't try to train these massive layers
pre_trained_model.trainable = False

# Build the model as before
pre_trained_model = tf.keras.Sequential([
  pre_trained_model,
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(192, activation='relu'),
  tf.keras.layers.Dense(num_classes, activation='softmax'),
])
pre_trained_model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])



87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
# Train the model
# Watch the training and validation accuracy over time
# A notably higher training accuracy means overfitting
epochs=5
history = pre_trained_model.fit(
  train,
  validation_data=validation,
  epochs=epochs
)

Epoch 1/5
23/23 [==============================] - 32s 1s/step - loss: 1.4218 - categorical_accuracy: 0.5116 - val_loss: 1.1410 - val_categorical_accuracy: 0.5559
Epoch 2/5
23/23 [==============================] - 26s 1s/step - loss: 0.8646 - categorical_accuracy: 0.6764 - val_loss: 0.9365 - val_categorical_accuracy: 0.6349
Epoch 3/5
23/23 [==============================] - 26s 1s/step - loss: 0.7029 - categorical_accuracy: 0.7388 - val_loss: 0.8458 - val_categorical_accuracy: 0.6921
Epoch 4/5
23/23 [==============================] - 26s 1s/step - loss: 0.5736 - categorical_accuracy: 0.7977 - val_loss: 0.8771 - val_categorical_accuracy: 0.6676
Epoch 5/5
23/23 [==============================] - 26s 1s/step - loss: 0.4299 - categorical_accuracy: 0.8522 - val_loss: 0.9493 - val_categorical_accuracy: 0.6649


In [ ]:
pre_trained_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 1, 1, 2048)        21802784  
                                                                 
 flatten_3 (Flatten)         (None, 2048)              0         
                                                                 
 dense_7 (Dense)             (None, 192)               393408    
                                                                 
 dense_8 (Dense)             (None, 5)                 965       
                                                                 
Total params: 22,197,157
Trainable params: 394,373
Non-trainable params: 21,802,784
_________________________________________________________________


### Questions
1. How did the network do? What were the final training and validation accuracies?

The network did pretty well. The final training accuracy was 85.2% and validation accuracy was 66.5%.

2. Is there evidence of overfitting?

There is evidence of overfitting because overfitting takes place when the validation accuracy is less then training accuracy. 


3. Why might you include pre-trained layers? Are they improving performance?

Pre trained layers eradicated the need to create a neural network every single time making the process more efficient and improving performance. 

4. How does the number of parameters (weights) compare to the previous networks (Hint: look at the summaries).

The number of parameters for this network is much higher than the number of parameters in the previous networks. 